In [1]:
# add parent directory to path: enable import from parent dir
import sys
sys.path.append('../')

from environment import SmartBrokerEnv
from agents.a2c import A2C
from networks.a2c.actor import Actor
from networks.a2c.critic import Critic

import gym
import matplotlib.pyplot as plt
import torch

In [2]:
norm_cols = []
cols = ['date'] + ['open', 'high', 'low', 'close', f'Volume XRP']
batch_dur = 30
env = SmartBrokerEnv(
    batch_dur=batch_dur,
    df_info={
        'start_date': '2021-01-01',
        'end_date': '2021-02-01',
        'norm_cols': norm_cols,
        'cols': cols,
    },
    portfolio={

    },
)

actor_model = Actor(
    state_dim=batch_dur*3+3,
    action_dim=3,
)
critic_model = Critic(state_dim=batch_dur*3+3)
a2c = A2C(
    env=env, 
    actor=actor_model,
    critic=critic_model,
    n_actns=3,
    actor_optmz=torch.optim.Adam(actor_model.parameters(), lr=0.001),
    critic_optmz=torch.optim.Adam(critic_model.parameters(), lr=0.001),
    hyprprms={
        'gamma': 0.9995,
    },
    log_freq=1,
    mdl_pth='../models/a2c_nn',
)

In [ ]:
a2c.run(1000)


Ep: 0 | TS: 19970 | L: 3127993304.056 | R: 17.74 | P: 19.87 | R.Avg P: 19.87 | NW: 119.87 | R.Avg NW: 119.87 | R.U: 340
Ep: 1 | TS: 39940 | L: 1453068673.938 | R: 23.75 | P: 17.01 | R.Avg P: 18.44 | NW: 117.01 | R.Avg NW: 118.44 | R.U: 264
Ep: 2 | TS: 59910 | L: 707255555.579 | R: 27.14 | P: 22.4 | R.Avg P: 19.76 | NW: 122.4 | R.Avg NW: 119.76 | R.U: 250
Ep: 3 | TS: 79880 | L: 706666499.55 | R: 24.66 | P: 15.1 | R.Avg P: 18.6 | NW: 115.1 | R.Avg NW: 118.6 | R.U: 244
Ep: 4 | TS: 99850 | L: 1011365098.563 | R: 22.89 | P: 12.69 | R.Avg P: 17.41 | NW: 112.69 | R.Avg NW: 117.41 | R.U: 247
Ep: 5 | TS: 119820 | L: 1192424583.352 | R: 22.37 | P: 12.8 | R.Avg P: 16.64 | NW: 112.8 | R.Avg NW: 116.64 | R.U: 249
Ep: 6 | TS: 139790 | L: 1132594773.695 | R: 28.24 | P: 25.59 | R.Avg P: 17.92 | NW: 125.59 | R.Avg NW: 117.92 | R.U: 247
Ep: 7 | TS: 159760 | L: 914251129.007 | R: 22.39 | P: 12.61 | R.Avg P: 17.26 | NW: 112.61 | R.Avg NW: 117.26 | R.U: 239
Ep: 8 | TS: 179730 | L: 658486888.132 | R: 19.08

In [ ]:
a2c.evaluate(start_dt='2021-01-02 07:00', duration=30)

## Visualizations

### Training 

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(16, 12))
r_avg_rewards = []
r_avg_profits = []
r_avg_bal = []
r_avg_units_held = []
r_avg_loss = []
r_avg_net_worth = []
count = 0

for _, log in a2c.logs.items():
    r_avg_rewards.append(log['r_avg_reward'])
    r_avg_profits.append(log['r_avg_profit'])
    r_avg_bal.append(log['r_avg_bal'])
    r_avg_units_held.append(log['r_avg_units_held'])
    r_avg_loss.append(log['r_avg_loss'])
    r_avg_net_worth.append(log['r_avg_net_worth'])
    count += 1

ax[0][0].plot(range(count), r_avg_loss)
ax[0][0].set_title('Rolling avg loss per episode')

ax[0][1].plot(range(count), r_avg_rewards)
ax[0][1].set_title('Rolling avg reward per episode')

ax[1][0].plot(range(count), r_avg_profits)
ax[1][0].set_title('Rolling avg profit per episode')

ax[1][1].plot(range(count), r_avg_units_held)
ax[1][1].set_title('Rolling avg units held per episode')

ax[2][0].plot(range(count), r_avg_net_worth)
ax[2][0].set_title('Rolling avg net worth per episode')

ax[2][1].plot(range(count), r_avg_bal)
ax[2][1].set_title('Rolling avg balance per episode')

In [ ]:
# import pickle
# with open("../pickles/a2c_logs.pickle","wb") as f:
#     pickle.dump(temp, f, pickle.HIGHEST_PROTOCOL)